In [2]:
# Import Required Python Base Packages
import sys
import unittest
import numpy as np
import random
import collections
import os
import time as tp
import glob

# Create a Safety Mechanism for Not Having MatPlotLib Installed
try:
    %matplotlib inline
    import matplotlib.pyplot as plt
    from matplotlib.pyplot import *
    from mpl_toolkits.mplot3d import Axes3D
    HAS_MATPLOTLIB = True
    
    matplotlib.rcParams['figure.figsize'] = (16, 6)
    matplotlib.rcParams['font.size'] = (14)
except ImportError:
    HAS_MATPLOTLIB = False

# Import the Amuse Base Packages
from amuse import datamodel
from amuse.units import nbody_system
from amuse.units import units
from amuse.units import constants
from amuse.datamodel import particle_attributes
from amuse.io import *
from amuse.lab import *

# Import the Amuse Stellar Packages
from amuse.ic.kingmodel import new_king_model
from amuse.ic.salpeter import new_salpeter_mass_distribution_nbody

# Import the Amuse Gravity & Close-Encounter Packages
from amuse.community.ph4.interface import ph4 as grav
from amuse.community.smalln.interface import SmallN
from amuse.community.kepler.interface import Kepler
from amuse.couple import multiples

# Importing cPickle/Pickle
try:
   import cPickle as pickle
except:
   import pickle

from tycho import read, util

In [5]:
# Import Encounter file
file_prefix = "Josh2"
file_directory = "/home/draco/jthornton/Tycho/Encounters/"
file_end = "_encounters.pkl"

encounterInfoReload = None
encounter_file = open(file_directory+file_prefix+file_end, "rb")
encounterInfoReload = pickle.load(encounter_file)
encounter_file.close()

In [59]:
def get_component_binary_elements(comp1, comp2, kep, peri = 0):

    mass = comp1.mass + comp2.mass
    pos = comp2.position - comp1.position
    vel = comp2.velocity - comp1.velocity
    kep.initialize_from_dyn(mass, pos[0], pos[1], pos[2],
                            vel[0], vel[1], vel[2])
    a,e = kep.get_elements()
    r = kep.get_separation()
    E,J = kep.get_integrals()           # per unit reduced mass, note
    if peri:
        M,th = kep.get_angles()
        if M < 0:
            kep.advance_to_periastron()
        else:
            kep.return_to_periastron()
    t = kep.get_time()
    P = kep.get_period()
    print kep.get_period()
    
    return mass,a,e,r,E,t,P

In [42]:
from collections import defaultdict

encounterInformation = defaultdict(list)
for key in encounterInfoReload:
    encounterInformation[key] = []
Mass = []
semimajor = []
eccentricity = []
Energy = []
Time = []
master_set, ic_array, converter = read.read_initial_state(file_prefix)
kep = Kepler(unit_converter=converter, redirection="none")
kep.initialize_code()

In [44]:
from tycho import create

num_stars = 1
num_planets = 0
IBF = 1
seed = 0
stars_SI, converter = create.king_cluster(num_stars, num_binaries=IBF, seed=seed)


In [52]:
num_star=1
num_planet=1
ibf=0

stars_SI2, converter2 = create.king_cluster(num_star, num_binaries=ibf, seed=seed)
systems_SI = create.planetary_systems(stars_SI2, converter2, num_planet, 'test_planets', Jupiter=True)
MasterSet = datamodel.Particles()
MasterSet.add_particles(stars_SI2)
MasterSet.add_particles(systems_SI)

In [66]:
try:
    kep2.stop()
    kep.stop()
except:
    pass
kep = Kepler(unit_converter=converter, redirection="none")
kep.initialize_code()

kep2 = Kepler(unit_converter=converter2, redirection="none")
kep2.initialize_code()

In [53]:
print MasterSet

                 key    host_star           id         mass       radius         type           vx           vy           vz            x            y            z
                   -         none         none           kg            m         none    m * s**-1    m * s**-1    m * s**-1            m            m            m
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========
  275342694097063007            0            1    9.039e+28    6.798e+12         star    0.000e+00    0.000e+00    0.000e+00    0.000e+00    0.000e+00    0.000e+00
 1200733817644779700            1      5000000    1.899e+27    4.551e+08       planet    1.928e+03    3.144e+03    1.184e+03    1.029e+12   -1.077e+11    3.876e+11
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========


In [45]:
print stars_SI

                 key           id         mass       radius         type           vx           vy           vz            x            y            z
                   -         none           kg           AU         none    m * s**-1    m * s**-1    m * s**-1            m            m            m
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========
18033081871020678442            2    9.424e+29    4.738e+02         star    2.848e+02    1.436e+03    3.322e+02    3.226e+12   -1.082e+12    7.465e+11
 6156527733378841055            3    5.385e+29    2.707e+02         star   -4.983e+02   -2.512e+03   -5.814e+02   -5.646e+12    1.894e+12   -1.306e+12
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========


In [70]:
mass,a,e,r,E,t,P = get_component_binary_elements(stars_SI[0], stars_SI[1],kep)

91005657886.0 s


In [68]:
mass,a,e,r,E,t,P = get_component_binary_elements(MasterSet[0],MasterSet[1],kep2)

5105531886.96 s


In [71]:
print mass
print stars_SI[0].mass+stars_SI[1].mass
print a
print e
print r
print E
print P.as_quantity_in(units.yr)

1.4808462008e+30 kg
1.4808462008e+30 kg
2.74724342534e+13 m
0.653687862156
9.5804601721e+12 m
-1798818.06066 m**2 * s**-2
2883.856872 yr


In [33]:
# Currently outputting 0 for all times in the oneset I've tried

# For loop to go through all keys in the dictionary
for key in encounterInfoReload:
    
    # try loop to cover for empty spaces in the dictionary for shorter runs
    try:
        i=0
        while i < len(encounterInfoReload[key])-2:
            
            # Method from Tyler's code
            mass,a,e,r,E,t = get_component_binary_elements(encounterInfoReload[key][i][0],encounterInfoReload[key][i+1][1],kep)
            # i +2 here to loop through the dictionary correctly (function takes 2 elements at a time)
            i+=2
            
            # Store the information in lists in the loop then append afterwards
            Mass.append(mass)
            semimajor.append(a)
            eccentricity.append(e)
            Energy.append(E)
            Time.append(t)
            
        # Store information in dictionary
        encounterInformation[key].append(Mass)
        encounterInformation[key].append(semimajor)
        encounterInformation[key].append(eccentricity)
        encounterInformation[key].append(Energy)
        encounterInformation[key].append(Time)
        
        # Clean the lists
        Mass = []
        semimajor = []
        eccentricity = []
        Energy = []
        Time = []
        
    except:
        pass
    